In [181]:
import streamlit as st
import gspread as gs 

gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
ss = gc.open_by_key(st.secrets['portfolio'].spreadsheet_key)
ws_operation = ss.worksheet('Operations')
# ws_deposit = ss.worksheet('Deposits')
ws_greenbull = ss.worksheet('GREENBULL')
ws_dict = ss.worksheet('Dict')

In [197]:
from typing import Dict
import pandas as pd

dict = pd.DataFrame(ws_dict.get_all_records())
assets: Dict[str,str] = dict[['Asset', 'Market', 'Currency']].set_index('Asset').to_dict()
depots: Dict[str,str] = dict[['Depot', 'Forex']].set_index('Depot').to_dict()
depots['Forex'][assets['Currency']['ESE']]

'EUREUR=X'

In [202]:
from typing import Dict
import yfinance as yf
import pandas as pd

greenbull = pd.DataFrame(ws_greenbull.get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
dict = pd.DataFrame(ws_dict.get_all_records())

# assets: Dict[str,str] = dict[['Asset', 'Market']].set_index('Asset').to_dict()['Market']
assets: Dict[str,str] = dict[['Asset', 'Market', 'Currency']].set_index('Asset').to_dict()
print(assets['Market'])
print(assets['Currency'])
market = yf.download(' '.join(list(assets['Market'].values())[:-1]), start='2021-04-01')['Close']
market = pd.concat([market, greenbull], axis=1)
market = pd.concat([market], keys=['Market'], axis=1)
market = pd.concat([market], keys=['Cotation'], axis=1)
# market = market.reorder_levels([1, 2, 0], axis=1)
# print(market.index)
# print(market.columns)

# depots: Dict[str,str] = dict[['Depot', 'Forex']].set_index('Depot').to_dict()['Forex']
depots: Dict[str,str] = dict[['Depot', 'Forex']].set_index('Depot').to_dict()
print(depots['Forex'])
forex: pd.DataFrame = yf.download(' '.join(list(depots['Forex'].values())), start='2021-04-01')['Close']
forex = pd.concat([forex], keys=['Forex'], axis=1)
forex = pd.concat([forex], keys=['Cotation'], axis=1)
# forex = forex.reorder_levels([1, 2, 0], axis=1)
# print(forex.index)
# print(forex.columns)

df = pd.concat([market, forex], axis=1).ffill().bfill()
print('\n')
print(df.index)
print(df.columns)
df.head()

{'ESE': 'ESE.PA', 'PUST': '', 'RS2K': '', 'OBLI': '', 'GOLD': 'GOLD.PA', 'BTC': 'BTC-USD', 'ETH': '', 'GREENBULL': 'GREENBULL'}
{'ESE': '€', 'PUST': '€', 'RS2K': '€', 'OBLI': '€', 'GOLD': '€', 'BTC': 'USDT', 'ETH': 'USDT', 'GREENBULL': '$'}
[*********************100%***********************]  3 of 3 completed
{'$': 'EURUSD=X', 'USDT': 'EURUSD=X', '€': 'EUREUR=X', '': ''}
[*********************100%***********************]  2 of 2 completed


DatetimeIndex(['2021-04-01', '2021-04-02', '2021-04-03', '2021-04-04',
               '2021-04-05', '2021-04-06', '2021-04-07', '2021-04-08',
               '2021-04-09', '2021-04-10',
               ...
               '2022-12-03', '2022-12-04', '2022-12-05', '2022-12-06',
               '2022-12-07', '2022-12-08', '2022-12-09', '2022-12-10',
               '2022-12-11', '2022-12-12'],
              dtype='datetime64[ns]', name='Date', length=621, freq='D')
MultiIndex([('Cotation', 'Market',   'BTC-USD'),
            ('Cotation', 'Market',    'ESE.P

Cotation                                                
                  Market                                 Forex          
                 BTC-USD     ESE.PA GOLD.PA GREENBULL EUREUR=X  EURUSD=X
Date                                                                    
2021-04-01  59095.808594  16.257601   58.57   14140.0      1.0  1.172883
2021-04-02  59384.312500  16.257601   58.57   14140.0      1.0  1.177995
2021-04-03  57603.890625  16.257601   58.57   14140.0      1.0  1.177995
2021-04-04  58758.554688  16.257601   58.57   14140.0      1.0  1.177995
2021-04-05  59057.878906  16.257601   58.57   14140.0      1.0  1.176817

In [221]:
import numpy as np

df = pd.concat([market, forex], axis=1).ffill().bfill()

ope = pd.DataFrame(ws_operation.get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
# print('ope')
# print(ope.index)
# print(ope.columns)
# print(ope.shape)


for portfolio in np.unique(ope['Portfolio']):
    dfp: pd.DataFrame = ope[ope['Portfolio'] == portfolio]
    print('', portfolio)
    # print(dfp.index)
    # print(dfp.columns)
    # print(dfp.shape)

    for asset in np.unique(dfp['Asset']):
        dfa: pd.DataFrame = dfp[dfp['Asset'] == asset]
        dfa = dfa.groupby('Date').agg({'Amount': "sum", 'Total': "sum"})#, 'Total€': "sum"})
        print('  ', asset)
        # print(dfa.index)
        # print(dfa.columns)
        # print(dfa.shape)

        amt, tot = pd.Series(dtype=float), pd.Series(dtype=float)
        for idx in dfa.index:
            amt[idx] = np.sum(dfa.loc[:idx]['Amount'])
            tot[idx] = np.sum(dfa.loc[:idx]['Total'])

        if asset in assets['Market'].keys():
            df = pd.concat([df, pd.DataFrame({
                ('Position', portfolio, asset): amt,
                ('Invested', portfolio, asset): tot,
            })], axis=1).ffill().fillna(0)

            df['Invested€', portfolio, asset] = df['Invested', portfolio, asset] * df['Cotation', 'Forex', depots['Forex'][assets['Currency'][asset]]]
            df['PRU', portfolio, asset] = df['Invested', portfolio, asset] / df['Position', portfolio, asset]
            df['Value', portfolio, asset] = df['Position', portfolio, asset] * df['Cotation', 'Market', assets['Market'][asset]]
            df['Value€', portfolio, asset] = df['Value', portfolio, asset] * df['Cotation', 'Forex', depots['Forex'][assets['Currency'][asset]]]
            df['PnL', portfolio, asset] = df['Value', portfolio, asset] - df['Invested', portfolio, asset]
            df['PnL€', portfolio, asset] = df['PnL', portfolio, asset] * df['Cotation', 'Forex', depots['Forex'][assets['Currency'][asset]]]

        elif asset in depots['Forex'].keys():
            df = pd.concat([df, pd.DataFrame({
                ('Amount', portfolio, asset): amt,
                ('Deposit', portfolio, asset): tot,
            })], axis=1).ffill().fillna(0)

            df['Deposit€', portfolio, asset] = df['Deposit', portfolio, asset] * df['Cotation', 'Forex', depots['Forex'][asset]]
            df['PnL', portfolio, asset] = df['Deposit', portfolio, asset] - df['Deposit€', portfolio, asset]
            df['PnL€', portfolio, asset] = df['PnL', portfolio, asset] * df['Cotation', 'Forex', depots['Forex'][asset]]
            
        

    if 'Deposit€' in df.columns and portfolio in df['Deposit€'].columns:
        df['Deposit€', portfolio, 'All'] = df['Deposit€', portfolio].sum(axis=1)
    if 'Invested€' in df.columns and portfolio in df['Invested€'].columns:
        df['Invested€', portfolio, 'All'] = df['Invested€', portfolio].sum(axis=1)
    if 'Value€' in df.columns and portfolio in df['Value€'].columns:
        df['Value€', portfolio, 'All'] = df['Value€', portfolio].sum(axis=1)
    if 'PnL€' in df.columns and portfolio in df['PnL€'].columns:
        df['PnL€', portfolio, 'All'] = df['PnL€', portfolio].sum(axis=1)


df = df.sort_index(axis=1)
print(df.index)
print(df.columns)

 DMA
   €
 ZEN
   $
   BTC
   ESE
   GOLD
   GREENBULL
   USDT
   €
DatetimeIndex(['2021-04-01', '2021-04-02', '2021-04-03', '2021-04-04',
               '2021-04-05', '2021-04-06', '2021-04-07', '2021-04-08',
               '2021-04-09', '2021-04-10',
               ...
               '2022-12-03', '2022-12-04', '2022-12-05', '2022-12-06',
               '2022-12-07', '2022-12-08', '2022-12-09', '2022-12-10',
               '2022-12-11', '2022-12-12'],
              dtype='datetime64[ns]', length=621, freq='D')
MultiIndex([(   'Amount',    'DMA',         '€'),
            (   'Amount',    'ZEN',         '$'),
            (   'Amount',    'ZEN',      'USDT'),
            (   'Amount',    'ZEN',         '€'),
            ( 'Cotation',  'Forex',  'EUREUR=X'),
            ( 'Cotation',  'Forex',  'EURUSD=X'),
            ( 'Cotation', 'Market',   'BTC-USD'),
            ( 'Cotation', 'Market',    'ESE.PA'),
            ( 'Cotation', 'Market',   'GOLD.PA'),
            ( 'Cotation', 'Marke

C:\Users\vcordier\AppData\Local\Temp\ipykernel_2544\2685852160.py:59: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Users\vcordier\AppData\Local\Temp\ipykernel_2544\2685852160.py:65: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Users\vcordier\AppData\Local\Temp\ipykernel_2544\2685852160.py:59: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Users\vcordier\AppData\Local\Temp\ipykernel_2544\2685852160.py:61: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Users\vcordier\AppData\Local\Temp\ipykernel_2544\2685852160.py:63: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Users\vcordier\AppData\Local\Temp\ipykernel_2544\2685852160.py:65: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [222]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Deposit€', 'ZEN', 'All'], name='Deposit€'))
fig.add_trace(go.Scatter(x=df.index, y=df['Invested€', 'ZEN', 'All'], name='Invested€'))
fig.add_trace(go.Scatter(x=df.index, y=df['Value€', 'ZEN', 'All'], name='Value€'))
fig.add_trace(go.Scatter(x=df.index, y=df['PnL€', 'ZEN', 'All'], name='PnL€'))
# for asset in df['PnL', 'ZEN'].columns:
#     print(asset)
#     fig.add_trace(go.Scatter(x=df.index, y=df['PnL', 'ZEN', asset], name=asset))

fig.show()

In [ ]:
tic = yf.download('SPY EURUSD=X BTC-USD ESE.PA GOLD.PA', start=min(portfolio['Date']))['Close'].ffill()
tic

[*********************100%***********************]  5 of 5 completed


,BTC-USD,ESE.PA,EURUSD=X,GOLD.PA,SPY
Date,,,,,
2021-04-30,57750.175781,16.617201,1.212709,58.556999,417.299988
2021-05-01,57828.050781,16.617201,1.212709,58.556999,417.299988
2021-05-02,56631.078125,16.617201,1.212709,58.556999,417.299988
2021-05-03,57200.292969,16.637800,1.202776,59.213001,418.200012
2021-05-04,53333.539062,16.475599,1.206127,59.227001,415.619995
...,...,...,...,...,...
2022-11-26,16464.281250,18.905701,1.040810,66.992996,402.329987
2022-11-27,16444.626953,18.905701,1.040810,66.992996,402.329987
2022-11-28,16217.322266,18.754200,1.036753,66.849998,395.910004


In [ ]:
import plotly.express as px

fig = px.line(tic[['ESE.PA', 'GOLD.PA']])
fig.show()